<a href="https://colab.research.google.com/github/woolozzx21/leesuan/blob/main/pytorch_cnn_model220103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets

transformation = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.1307),(0.3081,))])
train_dataset = datasets.MNIST('data/', train=True, transform=transformation, download=True)
test_dataset = datasets.MNIST('data/', train=False, transform=transformation, download=True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:
import matplotlib.pyplot as plt
def plot_image(image):
    image = image.numpy()[0]
    mean=0.1307
    std = 0.3081
    image = ((mean*image)+std)
    plt.imshow(image, cmap='gray')

In [ ]:
sample_data = next(iter(train_loader))
plot_image(sample_data[0][1])

In [ ]:
plot_image(sample_data[0][2])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320) #벡터로 핀다
        x = F.relu(self.fc1(x)) #풀리커넥티드로 실행
        x = F.dropout(x, training=self.training)
        x = self.fc2(x) #풀리커넥티드
        return F.log_softmax(x)  #


In [ ]:
def fit(device, epoch, model, data_loader, optimizer, phase='training'):
    if phase == 'training':
        model.train() # 신경망 훈련상태 (dropout적용)
    else:
        model.eval() # 신경망 검증 상태 
    running_loss = 0.0
    running_correct = 0.0
    for batch_idx, (data, target) in enumerate(data_loader): #(데이터, 목적)
        data, target = data.to(device), target.to(device)
        if phase == 'training':
            optimizer.zero_grad() # 모든 그라디언트 정보 날림
        output = model(data)
        loss = F.nll_loss(output, target) # 로스 계산
        running_loss +=  F.nll_loss(output, target, size_average=False).item()
        preds = output.data.max(dim=1, keepdim=True)[1] 
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum() # 같으면 true, 다른면 false
        if phase == 'training':
            loss.backward()
            optimizer.step()
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100.*running_correct/len(data_loader.dataset)
    print("Phase: {}, Loss: {}, Accuracy: {}".format(phase, loss, accuracy))
    return loss, accuracy


In [ ]:
import torch
import torch.optim as optim

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

model = Net()
model = model.to(DEVICE)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
train_loss, train_accuracy = [], []
test_loss, test_accuracy = [], []
for epoch in range(1, 21):
    epoch_loss, epoch_accuracy = fit(DEVICE, epoch, model, train_loader, optimizer, 'training')
    val_epoch_loss, val_epoch_accuracy = fit(DEVICE, epoch, model, test_loader, optimizer, 'validation')
    train_loss.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    test_loss.append(val_epoch_loss)
    test_accuracy.append(val_epoch_accuracy)


In [ ]:
plt.plot(range(1, len(train_accuracy)+1), train_accuracy, 'bo', label='training accuracy')
plt.plot(range(1, len(test_accuracy)+1), test_accuracy, 'r', label='test accuracy')
plt.legend()


In [ ]:
plt.plot(range(1, len(train_loss)+1), train_loss, 'bo', label='training loss')
plt.plot(range(1, len(test_loss)+1), test_loss, 'r', label='test loss')
plt.legend()
